# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Import Dependencies
import json
import requests
import gmaps
import os
from config import api_key
from pprint import pprint
import pandas as pd
from datetime import date
from datetime import timedelta
import requests
import urllib
from urllib.request import urlopen
import json
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
from citipy import citipy
from config import gkey
import warnings
warnings.filterwarnings('ignore')

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
input_file = "output_data/cities.csv"

weather_df = pd.read_csv(input_file,header=0)
weather_df.head()

,City,Country,Cloud_Cover,Latitude,Longitude,Wind_Speed,Humidity,Temp,Temp_Min,Temp_Max,Hemi
0,Ushuaia,AR,75.0,-54.8000,-68.3000,14.92,70.0,40.658,40.658,40.658,South
1,Nebbi,UG,98.0,2.4783,31.0889,1.81,63.0,74.264,74.264,74.264,North
2,Sitka,US,75.0,57.0531,-135.3300,8.23,55.0,33.692,33.692,33.692,North
3,Alta Floresta,BR,97.0,-9.8756,-56.0861,1.50,95.0,77.126,77.126,77.126,South
4,Ribeira Grande,PT,75.0,38.5167,-28.7000,4.63,77.0,61.196,61.196,61.196,North


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
coord = weather_df[["Latitude","Longitude"]]
humidity = weather_df["Humidity"]

In [66]:
# Plot Heatmap
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coord, weights=humidity, dissipating=False, max_intensity=50, point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
perfect_weather_df = weather_df.loc[(weather_df["Temp_Max"] > 70) & (weather_df["Temp_Max"] < 80) \
                             & (weather_df["Wind_Speed"] < 10) & (weather_df["Cloud_Cover"] == 0)]

perfect_weather_df

,City,Country,Cloud_Cover,Latitude,Longitude,Wind_Speed,Humidity,Temp,Temp_Min,Temp_Max,Hemi,Hotel
94,Saint-Pierre,RE,0.0,-21.3393,55.4781,2.57,70.0,75.074,73.040,75.290,South,
132,Gunjur,GM,0.0,13.2019,-16.7339,2.06,50.0,75.272,75.272,75.272,North,
223,Dubai,AE,0.0,25.2582,55.3047,2.57,68.0,72.842,71.852,77.216,North,
315,Broken Hill,AU,0.0,-31.9500,141.4333,8.01,46.0,70.628,70.628,70.628,South,
337,Praia,CV,0.0,14.9215,-23.5087,6.17,69.0,75.740,75.740,75.740,North,
367,Nouadhibou,MR,0.0,20.9310,-17.0347,7.20,29.0,75.182,75.182,75.182,North,
392,Upington,ZA,0.0,-28.4478,21.2561,2.06,16.0,71.978,71.978,71.978,South,
477,Stilfontein,ZA,0.0,-26.8449,26.7683,2.80,22.0,70.358,70.358,70.358,South,
483,Pisco,PE,0.0,-13.7000,-76.2167,4.12,69.0,75.254,71.024,75.254,South,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [57]:
hotel_df = perfect_weather_df[["City","Country", "Cloud_Cover", "Latitude", "Longitude", "Wind_Speed", "Humidity", "Temp","Temp_Min","Temp_Max","Hemi"]]

hotel_df['Hotel'] = ''
hotel_df.rename(columns = {'Latitude':'Lat'}, inplace = True)
hotel_df.rename(columns = {'Longitude':'Lng'}, inplace = True)
hotel_df

,City,Country,Cloud_Cover,Lat,Lng,Wind_Speed,Humidity,Temp,Temp_Min,Temp_Max,Hemi,Hotel
94,Saint-Pierre,RE,0.0,-21.3393,55.4781,2.57,70.0,75.074,73.040,75.290,South,
132,Gunjur,GM,0.0,13.2019,-16.7339,2.06,50.0,75.272,75.272,75.272,North,
223,Dubai,AE,0.0,25.2582,55.3047,2.57,68.0,72.842,71.852,77.216,North,
315,Broken Hill,AU,0.0,-31.9500,141.4333,8.01,46.0,70.628,70.628,70.628,South,
337,Praia,CV,0.0,14.9215,-23.5087,6.17,69.0,75.740,75.740,75.740,North,
367,Nouadhibou,MR,0.0,20.9310,-17.0347,7.20,29.0,75.182,75.182,75.182,North,
392,Upington,ZA,0.0,-28.4478,21.2561,2.06,16.0,71.978,71.978,71.978,South,
477,Stilfontein,ZA,0.0,-26.8449,26.7683,2.80,22.0,70.358,70.358,70.358,South,
483,Pisco,PE,0.0,-13.7000,-76.2167,4.12,69.0,75.254,71.024,75.254,South,


In [77]:
for ind in hotel_df.index:
    
    lat_lon = ''
    lat_lon = str(hotel_df['Lat'][ind])+','+ str(hotel_df['Lng'][ind])

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
                "location": lat_lon,
                "rankby": "distance",
                "type": "hotel",
                "keyword":"luxury hotel",
                "key": gkey,
        }

    response = requests.get(base_url, params=params).json()
    results = response['results']

    try:
        hotel_df['Hotel'][ind] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    
hotel_df

Missing field/result... skipping.


,City,Country,Cloud_Cover,Lat,Lng,Wind_Speed,Humidity,Temp,Temp_Min,Temp_Max,Hemi,Hotel
94,Saint-Pierre,RE,0.0,-21.3393,55.4781,2.57,70.0,75.074,73.040,75.290,South,Hôtel Le Saint-Pierre
132,Gunjur,GM,0.0,13.2019,-16.7339,2.06,50.0,75.272,75.272,75.272,North,"Footsteps Eco Lodge, The Gambia"
223,Dubai,AE,0.0,25.2582,55.3047,2.57,68.0,72.842,71.852,77.216,North,"Regent Palace Hotel, Dubai"
315,Broken Hill,AU,0.0,-31.9500,141.4333,8.01,46.0,70.628,70.628,70.628,South,Artmecca Holiday House
337,Praia,CV,0.0,14.9215,-23.5087,6.17,69.0,75.740,75.740,75.740,North,Hotel Santa Maria
367,Nouadhibou,MR,0.0,20.9310,-17.0347,7.20,29.0,75.182,75.182,75.182,North,
392,Upington,ZA,0.0,-28.4478,21.2561,2.06,16.0,71.978,71.978,71.978,South,Kalaharihuis Guest House
477,Stilfontein,ZA,0.0,-26.8449,26.7683,2.80,22.0,70.358,70.358,70.358,South,African Elephant’s Den Guesthouse Stilfontein
483,Pisco,PE,0.0,-13.7000,-76.2167,4.12,69.0,75.254,71.024,75.254,South,Hotel Presidente


In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [83]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)

# Display figure

fig


Figure(layout=FigureLayout(height='420px'))